# imports

In [1]:
library(tidyverse)
library(dplyr)
#install.packages("Metrics")
library(Metrics)
library(rpart)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1       v purrr   0.3.2  
v tibble  2.1.1       v dplyr   0.8.0.1
v tidyr   0.8.3       v stringr 1.4.0  
v readr   1.3.1       v forcats 0.4.0  
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'Metrics' was built under R version 3.6.3"

In [2]:
cl_data <- read_csv("../datasets/Dataset 3 Beijing/Datacleaning/cleaned_beijing.csv")
cl_data <- cl_data[ -c(1) ]

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  .default = col_double(),
  tradeTime = col_date(format = ""),
  buildingType = col_character(),
  renovationCondition = col_character(),
  buildingStructure = col_character(),
  elevator = col_character(),
  fiveYearsProperty = col_character(),
  subway = col_character(),
  district = col_character()
)
See spec(...) for full column specifications.


In [3]:
head(cl_data)

Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,1.111027e+12,2016-08-09,1464,106,415.0,31680,131.00,2,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,1.111027e+12,2016-07-28,903,126,575.0,43436,132.38,2,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,1.111041e+12,2016-12-11,1271,48,1030.0,52021,198.00,3,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,1.111043e+12,2016-09-30,965,138,297.5,22202,134.00,3,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,1.111027e+12,2016-08-28,927,286,392.0,48396,81.00,2,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,1.111027e+12,2016-07-22,861,57,275.6,52000,53.00,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [4]:
summary(cl_data)

      Lng             Lat             Cid              tradeTime         
 Min.   :116.1   Min.   :39.63   Min.   :1.111e+12   Min.   :2002-06-01  
 1st Qu.:116.3   1st Qu.:39.89   1st Qu.:1.111e+12   1st Qu.:2013-12-26  
 Median :116.4   Median :39.93   Median :1.111e+12   Median :2015-09-27  
 Mean   :116.4   Mean   :39.95   Mean   :1.126e+12   Mean   :2015-04-30  
 3rd Qu.:116.5   3rd Qu.:40.00   3rd Qu.:1.111e+12   3rd Qu.:2016-08-12  
 Max.   :116.7   Max.   :40.25   Max.   :1.185e+14   Max.   :2018-01-28  
      DOM            followers         totalPrice         price       
 Min.   :   1.00   Min.   :   0.00   Min.   :  13.0   Min.   : 10000  
 1st Qu.:   7.00   1st Qu.:   0.00   1st Qu.: 206.0   1st Qu.: 28362  
 Median :   7.00   Median :   5.00   Median : 295.0   Median : 39133  
 Mean   :  18.54   Mean   :  17.25   Mean   : 350.6   Mean   : 44003  
 3rd Qu.:   8.00   3rd Qu.:  19.00   3rd Qu.: 428.0   3rd Qu.: 54348  
 Max.   :1677.00   Max.   :1143.00   Max.   :4900.0   Ma

In [5]:
unique(cl_data$constructionTime)

[1] 2005 2004 2008 1960 1997 2009 1991 2001 1990 2011 2000 1998 2010 1996 1993
[16] 2006 2002 2012 1989 2003 2007 1994 1984 1992 2014 1985 1999 1979 1981 1976
[31] 1982 1975 1983 1986 1995 1965 2013 1988 1987 2015 1955 1980 1978 1958 1970
[46] 1956 1977 1964 1963 1967 2016 1974 1973 1959 1954 1962 1966 1957 1972 1971
[61] 1953 1968 1961 1950 1952 1969

removing  Cid and total price. cid has too many factors (computer crashes when trained using as factor) 

In [6]:
data = cl_data[-c(3,7)]

# Data After Modifiation

In [7]:
head(data)

Lng,Lat,tradeTime,DOM,followers,price,square,livingRoom,drawingRoom,kitchen,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,2016-08-09,1464,106,31680,131.00,2,1,1,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,2016-07-28,903,126,43436,132.38,2,2,1,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,2016-12-11,1271,48,52021,198.00,3,2,1,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,2016-09-30,965,138,22202,134.00,3,1,1,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,2016-08-28,927,286,48396,81.00,2,1,1,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,2016-07-22,861,57,52000,53.00,1,0,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [8]:
colnames(data)

[1] "Lng"                 "Lat"                 "tradeTime"          
 [4] "DOM"                 "followers"           "price"              
 [7] "square"              "livingRoom"          "drawingRoom"        
[10] "kitchen"             "bathRoom"            "floor"              
[13] "buildingType"        "constructionTime"    "renovationCondition"
[16] "buildingStructure"   "ladderRatio"         "elevator"           
[19] "fiveYearsProperty"   "subway"              "district"           
[22] "communityAverage"

In [9]:
str(data)

Classes 'tbl_df', 'tbl' and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : num  2 2 3 3 2 1 2 3 1 1 ...
 $ drawingRoom        : num  1 2 2 1 1 0 1 2 0 0 ...
 $ kitchen            : num  1 1 1 1 1 1 1 1 1 0 ...
 $ bathRoom           : num  1 2 3 1 1 1 1 2 1 0 ...
 $ floor              : num  26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ constructionTime   : num  2005 2004 2005 2008 1960 ...
 $ renovationCondition: chr  "Simplicit" "Hardcover" "Simplicit" "Other" ...
 $ buildingStructure  : chr  "Steel/Con

# Runing an analysis of variabce to see the significat factors

In [10]:
anova <- aov( price ~ . , data = data )

In [11]:
summary(anova)

                        Df    Sum Sq   Mean Sq  F value Pr(>F)    
Lng                      1 3.362e+12 3.362e+12  35190.1 <2e-16 ***
Lat                      1 2.696e+11 2.696e+11   2822.3 <2e-16 ***
tradeTime                1 3.956e+13 3.956e+13 414057.9 <2e-16 ***
DOM                      1 9.809e+11 9.809e+11  10266.3 <2e-16 ***
followers                1 1.049e+10 1.049e+10    109.8 <2e-16 ***
square                   1 3.485e+12 3.485e+12  36475.2 <2e-16 ***
livingRoom               1 3.904e+11 3.904e+11   4086.1 <2e-16 ***
drawingRoom              1 5.909e+10 5.909e+10    618.4 <2e-16 ***
kitchen                  1 1.609e+11 1.609e+11   1683.8 <2e-16 ***
bathRoom                 1 4.947e+11 4.947e+11   5178.2 <2e-16 ***
floor                    1 5.533e+11 5.533e+11   5791.1 <2e-16 ***
buildingType             3 5.894e+11 1.965e+11   2056.5 <2e-16 ***
constructionTime         1 5.432e+12 5.432e+12  56851.2 <2e-16 ***
renovationCondition      3 1.355e+12 4.517e+11   4727.3 <2e-16

# Spliting Data

In [12]:
train_index <- sample(1:nrow(data), 0.8 * nrow(data))
test_index <- setdiff(1:nrow(data), train_index)

# Build X_train, y_train, X_test, y_test
train <- data[train_index,]


test <- data[test_index,]


# Model creation

## model with linercombiantion of all facotors

In [13]:
model <- lm(price ~ . , data = train )

In [14]:
summary(model)


Call:
lm(formula = price ~ ., data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-90497  -6055   -600   5426 142777 

Coefficients:
                                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)                      2.686e+05  5.809e+04   4.624 3.77e-06 ***
Lng                             -3.208e+03  4.357e+02  -7.362 1.82e-13 ***
Lat                             -3.686e+03  4.996e+02  -7.379 1.61e-13 ***
tradeTime                        2.262e+01  5.154e-02 438.770  < 2e-16 ***
DOM                              3.592e+01  6.316e-01  56.874  < 2e-16 ***
followers                        8.755e+00  7.030e-01  12.453  < 2e-16 ***
square                          -6.056e+01  1.236e+00 -48.999  < 2e-16 ***
livingRoom                       1.122e+02  4.270e+01   2.629  0.00857 ** 
drawingRoom                      1.222e+03  5.139e+01  23.779  < 2e-16 ***
kitchen                         -8.828e+02  2.130e+02  -4.145 3.40e-05 ***
bathRoom                       

mean funtion not working had to imporvised now 

In [15]:
p = predict(model, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 9769.752

## model with limited factors for initial testing

In [16]:
model_limited <- lm(price ~ square + buildingType +constructionTime + subway + district , data = train )


In [17]:
summary(model_limited)


Call:
lm(formula = price ~ square + buildingType + constructionTime + 
    subway + district, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-75171 -11243  -3434   9115 113328 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)              78277.333  10487.504   7.464 8.43e-14 ***
square                     -30.529      1.106 -27.615  < 2e-16 ***
buildingTypePlate       -36439.404   2973.292 -12.256  < 2e-16 ***
buildingTypePlate/Tower -35698.590   2974.534 -12.001  < 2e-16 ***
buildingTypeTower       -38477.931   2974.069 -12.938  < 2e-16 ***
constructionTime            -3.250      5.056  -0.643 0.520296    
subwayNo_Subway          -3811.909     79.318 -48.059  < 2e-16 ***
districtChaoYang         12827.732    129.018  99.426  < 2e-16 ***
districtDaXing            1540.359    430.662   3.577 0.000348 ***
districtDongCheng        29373.899    197.911 148.420  < 2e-16 ***
districtFangShang        -1414.345    397.541  -3.558 0

In [18]:
p = predict(model_limited, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 17660.7

'price'
'square'
'livingRoom'
'drawingRoom'
'kitchen'
'bathRoom'
'floor'
'buildingType'
'constructionTime'
'renovationCondition'
'buildingStructure'
'ladderRatio'
'elevator'
'fiveYearsProperty'
'subway'
'district'## tree model

In [19]:
fit <- rpart(price ~ square + livingRoom + drawingRoom + kitchen + bathRoom + floor + buildingType + 
             constructionTime + renovationCondition + buildingStructure + elevator+ subway+ district, data = train ,method="anova")

In [20]:
summary(fit)

Call:
rpart(formula = price ~ square + livingRoom + drawingRoom + kitchen + 
    bathRoom + floor + buildingType + constructionTime + renovationCondition + 
    buildingStructure + elevator + subway + district, data = train, 
    method = "anova")
  n= 237080 

          CP nsplit rel error    xerror        xstd
1 0.23980338      0 1.0000000 1.0000042 0.004166674
2 0.06286745      1 0.7601966 0.7602083 0.002945616
3 0.05515125      2 0.6973292 0.6973434 0.002807699
4 0.03627643      3 0.6421779 0.6421946 0.002664773
5 0.01387834      4 0.6059015 0.6059226 0.002610417
6 0.01121155      5 0.5920231 0.5920507 0.002519412
7 0.01000000      6 0.5808116 0.5808424 0.002516809

Variable importance
           district renovationCondition   buildingStructure              subway 
                 68                  22                   2                   2 
              floor            elevator        buildingType    constructionTime 
                  2                   1                   

In [21]:
p = predict(fit, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 16563.51

In [28]:
save(fit, file = "models/limited_model.rda")

## Random forest

crashes computer uses all ram

## Extended liner Model

In [23]:
model_ext<- lm(price ~ square + livingRoom + drawingRoom + kitchen + bathRoom + floor + buildingType + 
             constructionTime + renovationCondition + buildingStructure + elevator+ subway+ district , data = train )

In [24]:
summary(model_ext)


Call:
lm(formula = price ~ square + livingRoom + drawingRoom + kitchen + 
    bathRoom + floor + buildingType + constructionTime + renovationCondition + 
    buildingStructure + elevator + subway + district, data = train)

Residuals:
   Min     1Q Median     3Q    Max 
-81164 -10689  -2521   7984 121451 

Coefficients:
                                  Estimate Std. Error  t value Pr(>|t|)    
(Intercept)                     273608.537  11879.595   23.032  < 2e-16 ***
square                             -84.862      1.992  -42.605  < 2e-16 ***
livingRoom                        1195.291     70.345   16.992  < 2e-16 ***
drawingRoom                       -193.268     84.344   -2.291   0.0219 *  
kitchen                           4621.144    351.271   13.156  < 2e-16 ***
bathRoom                          3016.804    117.411   25.694  < 2e-16 ***
floor                              132.518      8.404   15.768  < 2e-16 ***
buildingTypePlate               -27471.663   3665.404   -7.495 6.66e-1

In [25]:
p = predict(model_ext, test)
error = (p-test[6])
mean(as.matrix(error)^2)^.5

[1] 16190.09

In [27]:
save(model_ext, file = "models/extended_model.rda")